In [1]:
import os

import torch
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import albumentations as A

from going_modular.dataloader.magface import create_magface_dataloader
from going_modular.model.MagFaceRecognition import MagFaceRecognition
from going_modular.train_eval.magface.train import fit
from going_modular.loss.MagLoss import MagLoss
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint
from going_modular.utils.transforms import RandomResizedCropRect, GaussianNoise

device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)

CONFIGURATION = {
    # Thư mục
    'type': 'albedo',
    'train_dir': './Dataset/Albedo/train',
    'test_dir': './Dataset/Albedo/test',
    
    # Cấu hình train
    'backbone': 'iresnet18',
    'epochs': 1000,
    'num_workers': 4,
    'batch_size': 16,
    'image_size': 224,
    'num_class': len(os.listdir('./Dataset/Albedo/train')),
    'embedding_size': 512,
    
    'learning_rate': 0.2,
    'weight_decay': 5e-4,
    'momentum': 0.9,
    'alpha': 0.9,
    
    # Hàm m(ai) giúp thay đổi ai từ 0.25 đến 1.6
    'scale': 64,
    'lambda_g': 20,
    'l_margin': 0.45, 
    'u_margin': 0.8,
    'l_a': 10, 
    'u_a': 110,
}

train_transform = A.Compose([
    RandomResizedCropRect(256),
])

test_transform = A.Compose([
    A.Resize(height=CONFIGURATION['image_size'], width=CONFIGURATION['image_size'])
])

train_dataloader, test_dataloader = create_magface_dataloader(CONFIGURATION, train_transform, test_transform)

model = MagFaceRecognition(CONFIGURATION).to(device)
criterion = MagLoss(conf = CONFIGURATION)

optimizer = torch.optim.Adam(model.parameters(), lr=CONFIGURATION['learning_rate'])
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=40, T_mult=1, eta_min=1e-6)
checkpoint_path = os.path.abspath('checkpoint/magface/' + CONFIGURATION['type'] + '/models/checkpoint.pth')
modle_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1)
earlystop_dir = os.path.abspath('checkpoint/magface/' + CONFIGURATION['type'] + '/models')
early_stopping = MultiMetricEarlyStopping(
    monitor_keys=['test_cosine_auc', 'test_cosine_accuracy', 'test_euclidean_auc', 'test_euclidean_accuracy'],
    patience=40,
    mode='max',
    verbose=1,
    save_dir=earlystop_dir,
    start_from_epoch=30
)            

/home/vohoang/miniconda3/envs/torch/lib/python3.10/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.23 (you have 1.4.22). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
2024-12-19 16:55:16.710699: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-19 16:55:17.517582: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
fit(CONFIGURATION, 0, model, device, train_dataloader, test_dataloader, criterion, optimizer, scheduler, early_stopping, modle_checkpoint)

Success gen embedding: 2.6988301277160645s
Success caculate score: 0.07600831985473633
Finish: 0.11209964752197266
Success gen embedding: 0.8064663410186768s
Success caculate score: 0.0391237735748291
Finish: 0.0047376155853271484
Epoch 1:
	train: loss 3.560 | loss id   3.53 | top_1_acc 0.0067 | top_5_acc 0.0304 | cos_auc: 0.605 | cos_acc: 0.348 | eu_auc: 0.419 | eu_acc: 0.995
	val: cos_auc: 0.652cos_acc: 0.808 | eu_auc: 0.391 | eu_acc: 0.951
	Saving model and optimizer state to /media/vohoang/WorkSpace/ubuntu/projects/in-process/Bachelor-s-Project/checkpoint/magface/albedo/models/checkpoint.pth
Success gen embedding: 2.644620180130005s
Success caculate score: 0.01587986946105957
Finish: 0.1254897117614746
Success gen embedding: 0.7012531757354736s
Success caculate score: 0.040251731872558594
Finish: 0.004759311676025391
Epoch 2:
	train: loss 1.929 | loss id   1.91 | top_1_acc 0.0034 | top_5_acc 0.0191 | cos_auc: 0.605 | cos_acc: 0.498 | eu_auc: 0.401 | eu_acc: 0.988
	val: cos_auc: 0.6

KeyboardInterrupt: 

In [ ]:
checkpoint = torch.load('checkpoint/magface/' + CONFIGURATION['type'] + '/models/checkpoint.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
print(epoch)

In [ ]:
fit(CONFIGURATION, 246, model, device, train_dataloader, test_dataloader, criterion, optimizer, scheduler, early_stopping, modle_checkpoint)